In [ ]:
import os
import torch
import allennlp
from allennlp.data import Vocabulary, Instance
from allennlp.data.fields import TextField, LabelField
from simple_classifier.dataset_reader import YelpReviewJsonLinesReader
from allennlp.data.data_loaders import SimpleDataLoader

from simple_classifier.model import SimpleClassifier
from allennlp.modules import Embedding
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.seq2vec_encoders import BagOfEmbeddingsEncoder
from allennlp.models import Model

# Intro
This code is a simplified implementation of what happens during an AllenNLP training loop.
Normally, you will NOT need to write code like this, since the "train" command will 
handle all of it for you based on what's in your config file. But we are going to show you
the essence of what happens when you call the "train" command.

## Reading
Here, we'll instantiate the `DatasetReader` we made for our Yelp review dataset. 
Note that we will not pass it any arguments, and instead use its default values for `tokenizer` and `token_indexers`.
A `tokenizer` turns a string into a list of tokens. 
Don't worry about `token_indexers` for now--just know that it's used to turn tokens into integers.

In [ ]:
reader = YelpReviewJsonLinesReader()
print("Reader tokenizer:", reader.tokenizer)
print("Reader token indexers:", reader.token_indexers)

# Load our dev data and use list() to force it all into memory
instances = list(reader.read('data/dev_500.jsonl'))
sample_instance = instances[0]
print("\nExample instance:\n\n", sample_instance)

## Vocabulary Setup
Our vocabulary helps us keep in a single place our mappings between strings and integers. 
Vocabularies can be thought of very simply as nested dictionaries with a depth of 2: the first key is the **namespace**, which helps keep logically distinct string values (e.g. POS tags and tokens) distinct; the second key is either the token, in which case the final value will be the token's corresponding integer value, or the integer value, in which case the value is the token's string value.

In [ ]:
vocab = Vocabulary.from_instances(instances)
print("Vocabulary summary:\n\n", vocab)

print("Label mapping:\n", vocab.get_index_to_token_vocabulary("labels"))
print("\nFirst 10 token mappings:\n", {k: v for k, v in vocab.get_index_to_token_vocabulary("tokens").items() if k < 10})

print("\nIndex for \"dog\" in namespace \"tokens\":", vocab.get_token_index("dog", namespace="tokens"))
print("\nToken at index 540 in namespace \"tokens\":", vocab.get_token_from_index(540))

## Data Loading
The next step in a training pipeline is to create a `DataLoader` which will sit on top of a list of `Instance`s and manage turning them into tensors.
Note the complicated structure of `text`: don't worry about it for now, we'll handle it in our next section.

In [ ]:
loader = SimpleDataLoader(instances, batch_size=4, vocab=vocab)
batches = [batch for batch in loader]
sample_batch = batches[0]
print(sample_batch)

## Model Setup
Here we will load the model we trained in the previous section and use it to make predictions for our sample batch.
Note the format of the `probs` key in the output: there is one row per item in the batch, and every column corresponds to a particular value in the `"labels"` namespace in `vocab`.

In [ ]:
embedding_dim = 50
archive_path = 'model' + os.sep + 'model.tar.gz'
model = Model.from_archive(archive_path)

sample_output = model(text=sample_batch["text"], label=sample_batch["label"])
print(sample_output)

In [ ]:
# Exercise 13
# ==============================
new_instances = [
    reader.text_to_instance("Bad and slow service, burger tastes like cardboard", 0),
    reader.text_to_instance("The food here is always so yummy and our kid always has a great time!", 4)
]

def print_probs(instance):
    output = model.forward_on_instance(instance)
    class_probs = output["probs"]
    print("  Text:", instance["text"].tokens)
    print("  Label:", instance["label"].label) 
    for rating in ["1", "2", "3", "4", "5"]:
        index = vocab.get_token_index(rating, namespace="labels")
        print(f"  {rating} stars: {class_probs[index].item() * 100:.2f}%")

print("\nFirst instance")
print_probs(new_instances[0])
print("\nSecond instance")
print_probs(new_instances[1])


In [ ]:
# Exercise 14
# ==============================
vocab